In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

dataset_path = "asl_dataset"
img_width, img_height = 160, 160
num_classes = 36

# Create an ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

# Load and preprocess the training data
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=30,
    class_mode='categorical',
    subset='training'
)

# Load and preprocess the validation data
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=30,
    class_mode='categorical',
    subset='validation'
)

2023-05-20 14:48:51.311009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 2012 images belonging to 36 classes.
Found 502 images belonging to 36 classes.


In [2]:

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [3]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Save the trained model
model.save("sign_language_model.h5")

Epoch 1/10
67/67 [==============================] - 29s 428ms/step - loss: 2.3600 - accuracy: 0.3860 - val_loss: 0.8075 - val_accuracy: 0.7958
Epoch 2/10
67/67 [==============================] - 29s 434ms/step - loss: 0.7721 - accuracy: 0.7639 - val_loss: 0.4830 - val_accuracy: 0.8125
Epoch 3/10
67/67 [==============================] - 29s 431ms/step - loss: 0.3998 - accuracy: 0.8764 - val_loss: 0.5354 - val_accuracy: 0.8458
Epoch 4/10
67/67 [==============================] - 30s 445ms/step - loss: 0.3082 - accuracy: 0.9046 - val_loss: 0.5202 - val_accuracy: 0.8333
Epoch 5/10
67/67 [==============================] - 33s 494ms/step - loss: 0.1910 - accuracy: 0.9405 - val_loss: 0.4233 - val_accuracy: 0.8521
Epoch 6/10
67/67 [==============================] - 29s 426ms/step - loss: 0.1906 - accuracy: 0.9410 - val_loss: 0.4150 - val_accuracy: 0.8542
Epoch 7/10
67/67 [==============================] - 26s 391ms/step - loss: 0.1398 - accuracy: 0.9536 - val_loss: 0.3976 - val_accuracy: 0.8979

In [4]:
# Load the saved model
from keras.models import load_model

model = load_model("sign_language_model.h5")

# Load and preprocess the test data
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Found 502 images belonging to 36 classes.
16/16 [==============================] - 2s 137ms/step - loss: 0.4492 - accuracy: 0.8705
Test Loss: 0.44916149973869324
Test Accuracy: 0.870517909526825


In [5]:
# Predict labels for the test data
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Get the ground truth labels
true_labels = test_generator.classes

# Calculate precision, recall, and F1 score
from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

16/16 [==============================] - 2s 131ms/step
              precision    recall  f1-score   support

           0       0.08      0.07      0.08        14
           1       0.00      0.00      0.00        14
           2       0.05      0.07      0.06        14
           3       0.00      0.00      0.00        14
           4       0.00      0.00      0.00        14
           5       0.07      0.07      0.07        14
           6       0.00      0.00      0.00        14
           7       0.00      0.00      0.00        14
           8       0.00      0.00      0.00        14
           9       0.00      0.00      0.00        14
          10       0.00      0.00      0.00        13
          11       0.00      0.00      0.00        14
          12       0.07      0.07      0.07        14
          13       0.00      0.00      0.00        14
          14       0.00      0.00      0.00        14
          15       0.00      0.00      0.00        14
          16       0.00   